In [1]:
import pandas as pd

In [2]:
movies1=pd.read_csv("amazon_movies.csv")
movies2=pd.read_csv("netflix_movies.csv")

In [3]:
movies1.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,NaN,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...
3,s4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,"March 30, 2021",2014,NaN,69 min,Documentary,"Pink breaks the mold once again, bringing her ..."
4,s5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,"March 30, 2021",1989,NaN,45 min,"Drama, Fantasy",Teenage Matt Banting wants to work with a famo...


In [4]:
movies1.shape

(9668, 12)

In [5]:
movies2.shape

(8807, 12)

In [6]:
df1= pd.concat(map(pd.read_csv,['amazon_movies.csv', 'netflix_movies.csv']),ignore_index=True)

In [7]:
df1.describe()

,release_year
count,18475.000000
mean,2011.124980
std,15.262484
min,1920.000000
25%,2011.000000
50%,2017.000000
75%,2019.000000
max,2021.000000


In [8]:
df1.isna().sum()

show_id            0
type               0
title              0
director        4717
cast            2058
country         9827
date_added      9523
release_year       0
rating           341
duration           3
listed_in          0
description        0
dtype: int64

In [9]:
#df1.dropna(axis=0,inplace=True)

In [10]:
df1.isna().sum()

show_id            0
type               0
title              0
director        4717
cast            2058
country         9827
date_added      9523
release_year       0
rating           341
duration           3
listed_in          0
description        0
dtype: int64

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
data = df1['description']
data = data.astype(str)
feature = TfidfVectorizer(min_df=3,  max_features=None,
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3),
            stop_words = 'english')
data = feature.fit_transform(data).toarray().astype('int')

In [12]:
from sklearn.metrics.pairwise import sigmoid_kernel
sig = sigmoid_kernel(data, data)
print(sig[0])

[0.76159416 0.76159416 0.76159416 ... 0.76159416 0.76159416 0.76159416]


In [13]:
indices = pd.Series(movies1.index, index=movies1['title']).drop_duplicates()
def give_recomendations(title, sig=sig):
    # Get the index corresponding to original_title
    idx = indices[title]

    # Get the pairwsie similarity scores
    sig_scores = list(enumerate(sig[idx]))

    # Sort the movies
    sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)

    # Scores of the 10 most similar movies
    sig_scores = sig_scores[1:11]
    movie_indices = [i[0] for i in sig_scores]

    # Top 10 most similar movies
    return movies1['title'].iloc[movie_indices]


In [14]:
print(give_recomendations('Monster Maker'))

1      Take Care Good Night
2      Secrets of Deception
3        Pink: Staying True
4             Monster Maker
5     Living With Dinosaurs
6                 Hired Gun
7              Grease Live!
8           Global Meltdown
9            David's Mother
10           Forest Fairies
Name: title, dtype: object
